<a href="https://colab.research.google.com/github/rudycav/Web-Scraping-CoronaVirus-Cases/blob/master/Project/%20CoronaVirus_Cases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [538]:
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
from pandas import DataFrame
import pandas as pd
from datetime import datetime
import seaborn as sns
import numpy as np
import re
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

date = datetime.now()
date_time = date.strftime("%m/%d/%Y %H:%M:%S")

cases = []
data = []
header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36'}
url = requests.get('https://www.worldometers.info/coronavirus/', header)
bs = BeautifulSoup(url.content,'lxml')
title_numbers = bs.find_all(['h1','span'])
numbers = bs.find_all(class_='maincounter-number')
data_table = bs.find_all('table', class_='main_table_countries')

print('COVID-19 CORONAVIRUS PANDEMIC')
print('As of', date_time)
print()
print('Cases : ', title_numbers[5].text)
print('Deaths : ', title_numbers[7].text)
print('Recovered : ', title_numbers[9].text)

for table in data_table:
    headers = []
    rows = table.find_all('tr')
    for header in table.find('tr').find_all('th'):
        headers.append(header.text)
    for row in table.find_all('tr')[1:]:
        values = []
        for column in row.find_all(['th', 'td']):
            values.append(column.text)
        if values:
            dt = {headers[i]: values[i] for i in range(len(values))}
            data.append(dt)
            
df = pd.DataFrame(data)

df2 = df.rename(columns={"1stcase": "FirstCase", "Serious,Critical": "Critical"})

COVID-19 CORONAVIRUS PANDEMIC
As of 10/06/2022 03:14:23

Cases :  624,918,177 
Deaths :  6,555,411
Recovered :  604,737,533


In [539]:

def punctuation_removal(df):
    try:
        #removes N/A, commas, and + symbol, converts empty cells into 0s from the dataframe
        df = df.str.replace('N/A','').str.replace(',','').replace(r'^\s*$', np.nan, regex=True).replace(np.nan, 0).astype(float).astype(int)
    except:
        pass
    return df

df2 = df2.apply(punctuation_removal)


In [540]:
df2['Country,Other'] = df2['Country,Other'].replace(r'\n',' ', regex=True) 

In [541]:
df2.columns

Index(['#', 'Country,Other', 'TotalCases', 'NewCases', 'TotalDeaths',
       'NewDeaths', 'TotalRecovered', 'NewRecovered', 'ActiveCases',
       'Critical', 'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests',
       'Tests/\n1M pop\n', 'Population', 'Continent', '1 Caseevery X ppl',
       '1 Deathevery X ppl', '1 Testevery X ppl', 'New Cases/1M pop',
       'New Deaths/1M pop', 'Active Cases/1M pop'],
      dtype='object')

In [542]:
df2.head(10)

,#,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,Critical,...,TotalTests,Tests/\n1M pop\n,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl,New Cases/1M pop,New Deaths/1M pop,Active Cases/1M pop
0,0,North America,116665220,0,1542172,0,111708347,2623,3414701,7937,...,0,0,0,North America,0,0,0,0,0,0
1,0,Asia,190448953,28858,1481246,29,184655075,38456,4312632,10659,...,0,0,0,Asia,0,0,0,0,0,0
2,0,Europe,228582926,0,1922945,0,221677869,19447,4982112,9959,...,0,0,0,Europe,0,0,0,0,0,0
3,0,South America,64157352,0,1330345,0,62453197,862,373810,10214,...,0,0,0,South America,0,0,0,0,0,0
4,0,Oceania,12411438,1443,21034,15,12248217,0,142187,97,...,0,0,0,Australia/Oceania,0,0,0,0,0,0
5,0,Africa,12651567,0,257654,0,11994122,0,399791,1015,...,0,0,0,Africa,0,0,0,0,0,0
6,0,,721,0,15,0,706,0,0,0,...,0,0,0,,0,0,0,0,0,0
7,0,World,624918177,30301,6555411,44,604737533,61388,13625233,39881,...,0,0,0,All,0,0,0,0,0,0
8,1,USA,98411240,0,1086685,0,95432346,0,1892209,2733,...,1119730103,3344422,334805269,North America,3,308,0,0,0,5652
9,2,India,44603074,0,528733,0,44039883,0,34458,698,...,895958696,636953,1406631776,Asia,32,2660,2,0,0,24


In [562]:
df3 = df2.iloc[8:-8]

In [564]:
print(df3)

       #   Country,Other  TotalCases  NewCases  TotalDeaths  NewDeaths  \
8      1             USA    98411240         0      1086685          0   
9      2           India    44603074         0       528733          0   
10     3          France    35639699         0       155319          0   
11     4          Brazil    34743598         0       686640          0   
12     5         Germany    33652255         0       150289          0   
..   ...             ...         ...       ...          ...        ...   
725  226            Niue          80         0            0          0   
726  227    Vatican City          29         0            0          0   
727  228          Tuvalu          20         0            0          0   
728  229  Western Sahara          10         0            1          0   
729  230      MS Zaandam           9         0            2          0   

     TotalRecovered  NewRecovered  ActiveCases  Critical  ...  TotalTests  \
8          95432346             0 

In [543]:
df2.TotalCases.dtype

dtype('int64')

In [544]:
df2.TotalDeaths.dtype

dtype('int64')